<a href="https://colab.research.google.com/github/joshhowolahbi/ML-Secondary-School-Rankings-UK/blob/main/Component_One.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Component One

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn import metrics, neighbors, naive_bayes, preprocessing, linear_model
print('setup complete')

setup complete


* Load sheet “england_school_information” and reduce it to only the attributes: URN, LA, GENDER and RELCHAR

In [ ]:
esi_df = pd.read_excel('./england_school_information.xlsx')
esi_df.head()

,URN,LANAME,LA,ESTAB,LAESTAB,SCHNAME,STREET,LOCALITY,ADDRESS3,TOWN,POSTCODE,SCHSTATUS,OPENDATE,CLOSEDATE,MINORGROUP,SCHOOLTYPE,ISPRIMARY,ISSECONDARY,ISPOST16,AGELOW,AGEHIGH,GENDER,RELCHAR,ADMPOL
0,100000,City of London,201,3614,2013614,The Aldgate School,St James's Passage,Duke's Place,NaN,London,EC3A 5DE,Open,NaN,NaN,Maintained school,Voluntary aided school,1,0,0,3.0,11.0,Mixed,Church of England,Not applicable
1,100001,City of London,201,6005,2016005,City of London School for Girls,St Giles' Terrace,Barbican,NaN,London,EC2Y 8BB,Open,NaN,NaN,Independent school,Other independent school,1,1,1,7.0,18.0,Girls,None,Not applicable
2,100002,City of London,201,6006,2016006,St Paul's Cathedral School,2 New Change,NaN,NaN,London,EC4M 9AD,Open,NaN,NaN,Independent school,Other independent school,1,1,0,4.0,13.0,Mixed,Church of England,Not applicable
3,100003,City of London,201,6007,2016007,City of London School,107 Queen Victoria Street,NaN,NaN,London,EC4V 3AL,Open,NaN,NaN,Independent school,Other independent school,1,1,1,10.0,18.0,Boys,None,Not applicable
4,100008,Camden,202,2019,2022019,Argyle Primary School,Tonbridge Street,NaN,NaN,London,WC1H 9EG,Open,NaN,NaN,Maintained school,Community school,1,0,0,3.0,11.0,Mixed,Does not apply,Not applicable


In [ ]:
df = esi_df.loc[esi_df['ISSECONDARY'] == 1]
df = df[['URN', 'LA', 'GENDER', 'RELCHAR']]
df.head()

,URN,LA,GENDER,RELCHAR
1,100001,201,Girls,None
2,100002,201,Mixed,Church of England
3,100003,201,Boys,None
41,100049,202,Mixed,Does not apply
42,100050,202,Girls,Does not apply


In [ ]:
df.shape

(6396, 4)

* Load the sheet “england_census” and reduce it to the attributes URN, LA and all the percentage attributes

In [ ]:
ec_df = pd.read_excel('./england_census.xlsx')
ec_df.head(5)

,URN,LA,ESTAB,SCHOOLTYPE,NOR,NORG,NORB,PNORG,PNORB,TSENELSE,PSENELSE,TSENELK,PSENELK,NUMEAL,NUMENGFL,NUMUNCFL,PNUMEAL,PNUMENGFL,PNUMUNCFL,NUMFSM,NUMFSMEVER,PNUMFSMEVER
0,100000,201.0,3614,State-funded primary,285.0,141.0,144.0,49.5,50.5,5.0,1.8,50.0,17.5,141.0,143.0,1.0,49.5,50.2,0.4,32.0,38.0,16.2
1,100001,201.0,6005,Independent school,752.0,752.0,0.0,100.0,0.0,0.0,0.0,58.0,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100002,201.0,6006,Independent school,254.0,105.0,149.0,41.3,58.7,0.0,0.0,14.0,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100003,201.0,6007,Independent school,937.0,0.0,937.0,0.0,100.0,0.0,0.0,122.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005,202.0,1048,State-funded nursery,133.0,63.0,70.0,47.4,52.6,4.0,3.0,25.0,18.8,92.0,41.0,0.0,69.2,30.8,0.0,27.0,NaN,NaN


In [ ]:
ec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23942 entries, 0 to 23941
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   URN          23942 non-null  object 
 1   LA           23939 non-null  float64
 2   ESTAB        23942 non-null  object 
 3   SCHOOLTYPE   23942 non-null  object 
 4   NOR          23897 non-null  float64
 5   NORG         23897 non-null  float64
 6   NORB         23897 non-null  float64
 7   PNORG        23892 non-null  float64
 8   PNORB        23892 non-null  float64
 9   TSENELSE     23893 non-null  float64
 10  PSENELSE     23888 non-null  float64
 11  TSENELK      23895 non-null  float64
 12  PSENELK      23890 non-null  float64
 13  NUMEAL       21952 non-null  float64
 14  NUMENGFL     21952 non-null  float64
 15  NUMUNCFL     21952 non-null  float64
 16  PNUMEAL      21952 non-null  float64
 17  PNUMENGFL    21952 non-null  float64
 18  PNUMUNCFL    21952 non-null  float64
 19  NUMF

In [ ]:
df_2 = ec_df[['URN', 'LA', 'PNORG', 'PNORB', 'PSENELSE', 'PSENELK', 'PNUMEAL', 'PNUMENGFL', 'PNUMUNCFL', 'PNUMFSMEVER']]
df_2.shape

(23942, 10)

In [ ]:
pfsm_df = (df_2).copy()
pfsm_df.loc[:, 'PNUMFSM'] = round((ec_df['NUMFSM'] / ec_df['NOR']) * 100, 1)
pfsm_df.head()

,URN,LA,PNORG,PNORB,PSENELSE,PSENELK,PNUMEAL,PNUMENGFL,PNUMUNCFL,PNUMFSMEVER,PNUMFSM
0,100000,201.0,49.5,50.5,1.8,17.5,49.5,50.2,0.4,16.2,11.2
1,100001,201.0,100.0,0.0,0.0,7.7,NaN,NaN,NaN,NaN,NaN
2,100002,201.0,41.3,58.7,0.0,5.5,NaN,NaN,NaN,NaN,NaN
3,100003,201.0,0.0,100.0,0.0,13.0,NaN,NaN,NaN,NaN,NaN
4,100005,202.0,47.4,52.6,3.0,18.8,69.2,30.8,0.0,NaN,20.3


In [ ]:
pfsm_df.shape

(23942, 11)

⦁	Load the sheet “england_abs” and remove the attribute ESTAB

In [ ]:
eabs_df = pd.read_excel('./england_abs.xlsx')
eabs_df.head()

,LA,ESTAB,URN,PERCTOT,PPERSABS10
0,201,3614,100000.0,2.6,3.4
1,202,2000,136807.0,4.3,11
2,202,2001,139837.0,3.8,4.4
3,202,2002,140686.0,4.6,12.1
4,202,2019,100008.0,4.2,7.3


In [ ]:
eabs_df.shape

(21217, 5)

In [ ]:
new_eabs_df = eabs_df.drop('ESTAB', axis= 1)
new_eabs_df.head()

,LA,URN,PERCTOT,PPERSABS10
0,201,100000.0,2.6,3.4
1,202,136807.0,4.3,11
2,202,139837.0,3.8,4.4
3,202,140686.0,4.6,12.1
4,202,100008.0,4.2,7.3


In [ ]:
new_eabs_df.shape

(21217, 4)

* Load the sheet “england_ks4provisional” and reduce it to the attributes URN, LEA, TPUP, P8PUP, ATT8SCR, P8MEA, P8CILOW, P8CIUPP, EBACCAPS, PTL2BASICS_94, PTL2BASICS_95, PTEBACC_E_PTQ_EE, PTEBACC_94, PTEBACC_95, PT5EM_94 and P8_BANDING

In [ ]:
ks4_df = pd.read_excel('./england_ks4provisional.xlsx')
ks4_df.head()

,RECTYPE,LEA,ESTAB,URN,SCHNAME,ICLOSE,NFTYPE,TPUP,ATT8SCR,P8PUP,TP8ADJ,P8MEA,P8CILOW,P8CIUPP,P8MEA_ORIG,P8CILOW_ORIG,P8CIUPP_ORIG,PTL2BASICS_94,PTL2BASICS_95,EBACCAPS,PTEBACC_E_PTQ_EE,PTEBACC_94,PTEBACC_95,PT5EM_94,P8_BANDING
0,1,201.0,6007.0,100003.0,City of London School,0.0,IND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,201.0,6005.0,100001.0,City of London School for Girls,0.0,IND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,202.0,4285.0,100053.0,Acland Burghley School,0.0,CY,136.0,47.2,128,1,-0.08,-0.3,0.14,-0.09,-0.31,0.14,0.662,0.471,4.29,0.515,0.368,0.265,0.625,3
4,2,202.0,7137.0,100092.0,Camden Centre for Learning (CCfL) Special School,0.0,CYS,17.0,3.4,16,0,-2.98,-3.6,-2.35,-2.98,-3.62,-2.34,0.059,0,0.23,0,0,0,0,5


In [ ]:
new_ks4_df = ks4_df[['URN', 'LEA', 'TPUP', 'P8PUP', 'ATT8SCR', 'P8MEA', 'P8CILOW', 'P8CIUPP', 'EBACCAPS', 'PTL2BASICS_94', 'PTL2BASICS_95', 'PTEBACC_E_PTQ_EE', 'PTEBACC_94', 'PTEBACC_95', 'PT5EM_94', 'P8_BANDING']]
new_ks4_df.head()

,URN,LEA,TPUP,P8PUP,ATT8SCR,P8MEA,P8CILOW,P8CIUPP,EBACCAPS,PTL2BASICS_94,PTL2BASICS_95,PTEBACC_E_PTQ_EE,PTEBACC_94,PTEBACC_95,PT5EM_94,P8_BANDING
0,100003.0,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100001.0,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100053.0,202.0,136.0,128,47.2,-0.08,-0.3,0.14,4.29,0.662,0.471,0.515,0.368,0.265,0.625,3
4,100092.0,202.0,17.0,16,3.4,-2.98,-3.6,-2.35,0.23,0.059,0,0,0,0,0,5


In [ ]:
new_ks4_df.shape

(5652, 16)

* Load the sheet “england_ks4-pupdes” and reduce it to the attributes URN, LEA, OVERALL_DESTPER, NOT_SUSTAINEDPER, UNKNOWNPER, OVERALL_DESTPER_DIS, NOT_SUSTAINEDPER_DIS and UNKNOWNPER_DIS

In [ ]:
ks4p_df = pd.read_excel('./england_ks4-pupdest.xlsx')
ks4p_df.head()

,RECTYPE,LEA,ESTAB,URN,ICLOSE,SCHNAME,NFTYPE,COHORT,OVERALL_DEST,APPREN,EMPLOYMENT,EDUCATION,FE,SCH_6TH,SIXTH_COL,OTHER_EDU,NOT_SUSTAINED,UNKNOWN,OVERALL_DESTPER,APPRENPER,EMPLOYMENTPER,EDUCATIONPER,FEPER,SCH_6THPER,SIXTH_COLPER,OTHER_EDUPER,NOT_SUSTAINEDPER,UNKNOWNPER,COHORT_DIS,OVERALL_DEST_DIS,APPREN_DIS,EMPLOYMENT_DIS,EDUCATION_DIS,FE_DIS,SCH_6TH_DIS,SIXTH_COL_DIS,OTHER_EDU_DIS,NOT_SUSTAINED_DIS,UNKNOWN_DIS,OVERALL_DESTPER_DIS,...,EDUCATIONPER_DIS,FEPER_DIS,SCH_6THPER_DIS,SIXTH_COLPER_DIS,OTHER_EDUPER_DIS,NOT_SUSTAINEDPER_DIS,UNKNOWNPER_DIS,COHORT_NONDIS,OVERALL_DEST_NONDIS,APPREN_NONDIS,EMPLOYMENT_NONDIS,EDUCATION_NONDIS,FE_NONDIS,SCH_6TH_NONDIS,SIXTH_COL_NONDIS,OTHER_EDU_NONDIS,NOT_SUSTAINED_NONDIS,UNKNOWN_NONDIS,OVERALL_DESTPER_NONDIS,APPRENPER_NONDIS,EMPLOYMENTPER_NONDIS,EDUCATIONPER_NONDIS,FEPER_NONDIS,SCH_6THPER_NONDIS,SIXTH_COLPER_NONDIS,OTHER_EDUPER_NONDIS,NOT_SUSTAINEDPER_NONDIS,UNKNOWNPER_NONDIS,COHORT_18,OVERALL_DESTPER_18,COHORT_DIS_18,OVERALL_DESTPER_DIS_18,COHORT_NONDIS_18,OVERALL_DESTPER_NONDIS_18,COHORT_17,OVERALL_DESTPER_17,COHORT_DIS_17,OVERALL_DESTPER_DIS_17,COHORT_NONDIS_17,OVERALL_DESTPER_NONDIS_17
0,1,202.0,4000.0,137181.0,0.0,The UCL Academy,AC,161,152,1,2,149,28,119,1,1,8,1,0.94,0.01,0.01,0.93,0.17,0.74,0.01,0.01,0.05,0.01,86,80,0,1,79,18,59,SUPP,SUPP,5,1,0.93,...,0.92,0.21,0.69,SUPP,SUPP,0.06,0.01,75,72,1,1,70,10,60,SUPP,SUPP,3,0,0.96,0.01,0.01,0.93,0.13,0.8,SUPP,SUPP,0.04,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,202.0,4104.0,100049.0,0.0,Haverstock School,CY,199,180,1,3,176,67,100,6,3,18,1,0.9,0.01,0.02,0.88,0.34,0.5,0.03,0.02,0.09,0.01,123,109,0,2,107,37,63,4,3,13,1,0.89,...,0.87,0.3,0.51,0.03,0.02,0.11,0.01,76,71,1,1,69,30,37,2,0,5,0,0.93,0.01,0.01,0.91,0.39,0.49,0.03,0,0.07,0,207.0,0.92,156,0.9,51,0.98,208.0,0.88,147,0.87,61,0.92
2,1,202.0,4166.0,100050.0,0.0,Parliament Hill School,CY,172,161,0,0,161,26,126,7,2,7,4,0.94,0,0,0.94,0.15,0.73,0.04,0.01,0.04,0.02,92,87,0,0,87,17,65,3,2,4,1,0.95,...,0.95,0.18,0.71,0.03,0.02,0.04,0.01,80,74,0,0,74,9,61,4,0,3,3,0.93,0,0,0.93,0.11,0.76,0.05,0,0.04,0.04,175.0,0.95,95,0.95,80,0.96,178.0,0.96,92,0.95,86,0.97
3,1,202.0,4196.0,100051.0,0.0,Regent High School,CY,110,95,2,1,92,30,59,3,0,9,6,0.86,0.02,0.01,0.84,0.27,0.54,0.03,0,0.08,0.05,67,57,1,1,55,19,35,1,0,6,4,0.85,...,0.82,0.28,0.52,0.01,0,0.09,0.06,43,38,1,0,37,11,24,2,0,3,2,0.88,0.02,0,0.86,0.26,0.56,0.05,0,0.07,0.05,137.0,0.85,89,0.82,48,0.9,127.0,0.9,91,0.87,36,0.97
4,1,202.0,4275.0,100052.0,0.0,Hampstead School,CY,201,182,1,2,179,38,118,21,2,17,2,0.91,SP,0.01,0.89,0.19,0.59,0.1,0.01,0.08,0.01,101,88,1,2,85,25,53,6,1,13,0,0.87,...,0.84,0.25,0.52,0.06,0.01,0.13,0,100,94,0,0,94,13,65,15,1,4,2,0.94,0,0,0.94,0.13,0.65,0.15,0.01,0.04,0.02,206.0,0.93,117,0.89,89,0.98,200.0,0.94,117,0.91,83,0.98


In [ ]:
new_ks4p_df = ks4p_df[[ 'URN', 'LEA', 'OVERALL_DESTPER', 'NOT_SUSTAINEDPER', 'UNKNOWNPER', 'OVERALL_DESTPER_DIS', 'NOT_SUSTAINEDPER_DIS', 'UNKNOWNPER_DIS']]
new_ks4p_df.head()

,URN,LEA,OVERALL_DESTPER,NOT_SUSTAINEDPER,UNKNOWNPER,OVERALL_DESTPER_DIS,NOT_SUSTAINEDPER_DIS,UNKNOWNPER_DIS
0,137181.0,202.0,0.94,0.05,0.01,0.93,0.06,0.01
1,100049.0,202.0,0.9,0.09,0.01,0.89,0.11,0.01
2,100050.0,202.0,0.94,0.04,0.02,0.95,0.04,0.01
3,100051.0,202.0,0.86,0.08,0.05,0.85,0.09,0.06
4,100052.0,202.0,0.91,0.08,0.01,0.87,0.13,0


In [ ]:
new_ks4p_df.shape

(3877, 8)

* Merging all dataframes

In [ ]:
mrg_df = pd.merge(df, pfsm_df, on= ('URN', 'LA'), how='left')
mrg_df.head()

,URN,LA,GENDER,RELCHAR,PNORG,PNORB,PSENELSE,PSENELK,PNUMEAL,PNUMENGFL,PNUMUNCFL,PNUMFSMEVER,PNUMFSM
0,100001,201,Girls,None,100.0,0.0,0.0,7.7,NaN,NaN,NaN,NaN,NaN
1,100002,201,Mixed,Church of England,41.3,58.7,0.0,5.5,NaN,NaN,NaN,NaN,NaN
2,100003,201,Boys,None,0.0,100.0,0.0,13.0,NaN,NaN,NaN,NaN,NaN
3,100049,202,Mixed,Does not apply,41.4,58.6,3.2,11.8,73.1,26.8,0.1,64.5,41.2
4,100050,202,Girls,Does not apply,96.0,4.0,1.3,10.2,38.1,60.9,0.9,49.6,26.9


In [ ]:
mrg_df.shape

(6396, 13)

In [ ]:
mrg2_df = pd.merge(mrg_df, new_eabs_df, on= ('URN', 'LA'), how='left')
mrg2_df.shape

(6396, 15)

In [ ]:
mrg3_df = pd.merge(mrg2_df, new_ks4_df, on= 'URN', how='left')
mrg3_df.shape

(6396, 30)

In [ ]:
fnl_mrg_df = pd.merge(mrg3_df, new_ks4p_df, on= ('URN', 'LEA'), how='left')
fnl_mrg_df.shape

(6396, 36)

In [ ]:
fnl_mrg_df.head(20)

,URN,LA,GENDER,RELCHAR,PNORG,PNORB,PSENELSE,PSENELK,PNUMEAL,PNUMENGFL,PNUMUNCFL,PNUMFSMEVER,PNUMFSM,PERCTOT,PPERSABS10,LEA,TPUP,P8PUP,ATT8SCR,P8MEA,P8CILOW,P8CIUPP,EBACCAPS,PTL2BASICS_94,PTL2BASICS_95,PTEBACC_E_PTQ_EE,PTEBACC_94,PTEBACC_95,PT5EM_94,P8_BANDING,OVERALL_DESTPER,NOT_SUSTAINEDPER,UNKNOWNPER,OVERALL_DESTPER_DIS,NOT_SUSTAINEDPER_DIS,UNKNOWNPER_DIS
0,100001,201,Girls,None,100.0,0.0,0.0,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,201,Mixed,Church of England,41.3,58.7,0.0,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100003,201,Boys,None,0.0,100.0,0.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100049,202,Mixed,Does not apply,41.4,58.6,3.2,11.8,73.1,26.8,0.1,64.5,41.2,6.9,18.8,202.0,199.0,178,42.4,-0.09,-0.28,0.09,3.72,0.588,0.432,0.241,0.191,0.141,0.533,3,0.9,0.09,0.01,0.89,0.11,0.01
4,100050,202,Girls,Does not apply,96.0,4.0,1.3,10.2,38.1,60.9,0.9,49.6,26.9,5.3,12.5,202.0,175.0,164,58.6,0.77,0.58,0.97,5.33,0.76,0.577,0.577,0.417,0.326,0.749,1,0.94,0.04,0.02,0.95,0.04,0.01
5,100051,202,Mixed,Does not apply,45.0,55.0,3.6,11.1,82.6,17.4,0.0,69.0,43.0,6.7,19.2,202.0,154.0,139,42.1,-0.18,-0.4,0.03,3.77,0.591,0.403,0.604,0.201,0.071,0.526,3,0.86,0.08,0.05,0.85,0.09,0.06
6,100052,202,Mixed,Does not apply,47.5,52.5,2.4,6.9,55.8,44.2,0.0,54.2,22.8,5.4,14,202.0,200.0,188,44.6,-0.06,-0.24,0.13,4.08,0.63,0.315,0.585,0.24,0.12,0.57,3,0.91,0.08,0.01,0.87,0.13,0
7,100053,202,Mixed,Does not apply,33.7,66.3,6.2,13.4,28.2,70.3,1.5,47.1,26.2,6.3,18.5,202.0,136.0,128,47.2,-0.08,-0.3,0.14,4.29,0.662,0.471,0.515,0.368,0.265,0.625,3,0.95,0.04,0.01,0.9,0.07,0.03
8,100054,202,Girls,None,85.8,14.2,1.5,17.1,36.9,62.3,0.8,36.6,12.9,3.9,3.2,202.0,107.0,95,63.4,0.74,0.48,1,5.83,0.888,0.748,0.607,0.551,0.402,0.888,1,0.95,0.04,0.02,0.93,0.05,0.02
9,100055,202,Mixed,Roman Catholic,58.8,41.2,3.5,8.9,52.2,47.8,0.0,54.5,29.3,5.3,14.3,202.0,115.0,107,48.3,0.09,-0.15,0.34,4.56,0.67,0.452,0.8,0.409,0.261,0.609,3,0.99,0,0.01,0.98,0,0.02


In [ ]:
missing_values = fnl_mrg_df.isnull().sum()
missing_values

URN                        0
LA                         0
GENDER                     0
RELCHAR                  143
PNORG                    864
PNORB                    864
PSENELSE                 868
PSENELK                  866
PNUMEAL                 2239
PNUMENGFL               2239
PNUMUNCFL               2239
PNUMFSMEVER             2241
PNUMFSM                 2239
PERCTOT                 2226
PPERSABS10              2226
LEA                      901
TPUP                    2448
P8PUP                   2448
ATT8SCR                 2448
P8MEA                   2448
P8CILOW                 2448
P8CIUPP                 2448
EBACCAPS                2448
PTL2BASICS_94           2448
PTL2BASICS_95           2448
PTEBACC_E_PTQ_EE        2448
PTEBACC_94              2448
PTEBACC_95              2448
PT5EM_94                2448
P8_BANDING              2863
OVERALL_DESTPER         2668
NOT_SUSTAINEDPER        2668
UNKNOWNPER              2668
OVERALL_DESTPER_DIS     2668
NOT_SUSTAINEDP

* The **LA** and **LEA** column stands for the **Local Authority Number**, however from inspection, the LEA column has more missing values compared with LA therefore we are going to be dropping the LEA column for a clean data with no duplicates

In [ ]:
final_mrg_df = fnl_mrg_df.drop('LEA', axis=1)

In [ ]:
final_mrg_df.head()

,URN,LA,GENDER,RELCHAR,PNORG,PNORB,PSENELSE,PSENELK,PNUMEAL,PNUMENGFL,PNUMUNCFL,PNUMFSMEVER,PNUMFSM,PERCTOT,PPERSABS10,TPUP,P8PUP,ATT8SCR,P8MEA,P8CILOW,P8CIUPP,EBACCAPS,PTL2BASICS_94,PTL2BASICS_95,PTEBACC_E_PTQ_EE,PTEBACC_94,PTEBACC_95,PT5EM_94,P8_BANDING,OVERALL_DESTPER,NOT_SUSTAINEDPER,UNKNOWNPER,OVERALL_DESTPER_DIS,NOT_SUSTAINEDPER_DIS,UNKNOWNPER_DIS
0,100001,201,Girls,None,100.0,0.0,0.0,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,201,Mixed,Church of England,41.3,58.7,0.0,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100003,201,Boys,None,0.0,100.0,0.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100049,202,Mixed,Does not apply,41.4,58.6,3.2,11.8,73.1,26.8,0.1,64.5,41.2,6.9,18.8,199.0,178,42.4,-0.09,-0.28,0.09,3.72,0.588,0.432,0.241,0.191,0.141,0.533,3,0.9,0.09,0.01,0.89,0.11,0.01
4,100050,202,Girls,Does not apply,96.0,4.0,1.3,10.2,38.1,60.9,0.9,49.6,26.9,5.3,12.5,175.0,164,58.6,0.77,0.58,0.97,5.33,0.76,0.577,0.577,0.417,0.326,0.749,1,0.94,0.04,0.02,0.95,0.04,0.01


In [ ]:
final_mrg_df.shape

(6396, 35)

* Merge the combined data with the OFSTED ratings data in sheet “OFSTED Ratings”

In [ ]:
ofst_df = pd.read_excel('./OFSTED Ratings.xlsx')
ofst_df.head()

,Rating,URN
0,Unknown,100001
1,Unknown,100002
2,Unknown,100003
3,Good,100049
4,Good,100050


In [ ]:
combined_df = pd.merge(final_mrg_df, ofst_df, on= 'URN', how='left')
combined_df.head()

,URN,LA,GENDER,RELCHAR,PNORG,PNORB,PSENELSE,PSENELK,PNUMEAL,PNUMENGFL,PNUMUNCFL,PNUMFSMEVER,PNUMFSM,PERCTOT,PPERSABS10,TPUP,P8PUP,ATT8SCR,P8MEA,P8CILOW,P8CIUPP,EBACCAPS,PTL2BASICS_94,PTL2BASICS_95,PTEBACC_E_PTQ_EE,PTEBACC_94,PTEBACC_95,PT5EM_94,P8_BANDING,OVERALL_DESTPER,NOT_SUSTAINEDPER,UNKNOWNPER,OVERALL_DESTPER_DIS,NOT_SUSTAINEDPER_DIS,UNKNOWNPER_DIS,Rating
0,100001,201,Girls,None,100.0,0.0,0.0,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
1,100002,201,Mixed,Church of England,41.3,58.7,0.0,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
2,100003,201,Boys,None,0.0,100.0,0.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
3,100049,202,Mixed,Does not apply,41.4,58.6,3.2,11.8,73.1,26.8,0.1,64.5,41.2,6.9,18.8,199.0,178,42.4,-0.09,-0.28,0.09,3.72,0.588,0.432,0.241,0.191,0.141,0.533,3,0.9,0.09,0.01,0.89,0.11,0.01,Good
4,100050,202,Girls,Does not apply,96.0,4.0,1.3,10.2,38.1,60.9,0.9,49.6,26.9,5.3,12.5,175.0,164,58.6,0.77,0.58,0.97,5.33,0.76,0.577,0.577,0.417,0.326,0.749,1,0.94,0.04,0.02,0.95,0.04,0.01,Good


In [ ]:
combined_df.shape

(6396, 36)

In [ ]:
cmb_ofs_df = pd.read_excel('./Combined OFSTED data.xlsx')
cmb_ofs_df.head(30)

,URN,LA,GENDER,RELCHAR,PNORG,PNORB,PSENELSE,PSENELK,PNUMEAL,PNUMENGFL,PNUMUNCFL,PNUMFSM,PNUMFSMEVER,PERCTOT,PPERSABS10,TPUP,P8PUP,ATT8SCR,P8MEA,P8CILOW,P8CIUPP,EBACCAPS,PTL2BASICS_94,PTL2BASICS_95,PTEBACC_E_PTQ_EE,PTEBACC_94,PTEBACC_95,PT5EM_94,P8_BANDING,OVERALL_DESTPER,NOT_SUSTAINEDPER,UNKNOWNPER,OVERALL_DESTPER_DIS,NOT_SUSTAINEDPER_DIS,UNKNOWNPER_DIS,Rating
0,100001,201,Girls,None,100.0,0.0,0.0,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
1,100002,201,Mixed,Church of England,41.3,58.7,0.0,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
2,100003,201,Boys,None,0.0,100.0,0.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
3,100049,202,Mixed,Does not apply,41.4,58.6,3.2,11.8,73.1,26.8,0.1,0.411579,64.5,6.9,18.8,199.0,178,42.4,-0.09,-0.28,0.09,3.72,0.588,0.432,0.241,0.191,0.141,0.533,3,0.9,0.09,0.01,0.89,0.11,0.01,Good
4,100050,202,Girls,Does not apply,96.0,4.0,1.3,10.2,38.1,60.9,0.9,0.269231,49.6,5.3,12.5,175.0,164,58.6,0.77,0.58,0.97,5.33,0.76,0.577,0.577,0.417,0.326,0.749,1,0.94,0.04,0.02,0.95,0.04,0.01,Good
5,100051,202,Mixed,Does not apply,45.0,55.0,3.6,11.1,82.6,17.4,0.0,0.429711,69.0,6.7,19.2,154.0,139,42.1,-0.18,-0.4,0.03,3.77,0.591,0.403,0.604,0.201,0.071,0.526,3,0.86,0.08,0.05,0.85,0.09,0.06,Good
6,100052,202,Mixed,Does not apply,47.5,52.5,2.4,6.9,55.8,44.2,0.0,0.227829,54.2,5.4,14,200.0,188,44.6,-0.06,-0.24,0.13,4.08,0.63,0.315,0.585,0.24,0.12,0.57,3,0.91,0.08,0.01,0.87,0.13,0,Good
7,100053,202,Mixed,Does not apply,33.7,66.3,6.2,13.4,28.2,70.3,1.5,0.261905,47.1,6.3,18.5,136.0,128,47.2,-0.08,-0.3,0.14,4.29,0.662,0.471,0.515,0.368,0.265,0.625,3,0.95,0.04,0.01,0.9,0.07,0.03,Good
8,100054,202,Girls,None,85.8,14.2,1.5,17.1,36.9,62.3,0.8,0.128780,36.6,3.9,3.2,107.0,95,63.4,0.74,0.48,1,5.83,0.888,0.748,0.607,0.551,0.402,0.888,1,0.95,0.04,0.02,0.93,0.05,0.02,Outstanding
9,100055,202,Mixed,Roman Catholic,58.8,41.2,3.5,8.9,52.2,47.8,0.0,0.292546,54.5,5.3,14.3,115.0,107,48.3,0.09,-0.15,0.34,4.56,0.67,0.452,0.8,0.409,0.261,0.609,3,0.99,0,0.01,0.98,0,0.02,Good


In [ ]:
cmb_ofs_df.shape

(6396, 36)

* Comparing final combined data with the "Combined OFSTED data"

In [ ]:
combined_df.equals(cmb_ofs_df)

False

The above returned a false because from inspection the "percentage free school meal" **PNUMFSM** in our cleaned data **combined_df** is already in percentage while the values in the downloaded data **cmd_ofs_df** wasn't fully converted into percentage values.

checking for missing data in each attribute of the **Combined OFSTED data**

In [ ]:
missing_data = cmb_ofs_df.isnull().sum()
missing_data

URN                        0
LA                         0
GENDER                     0
RELCHAR                  143
PNORG                    864
PNORB                    864
PSENELSE                 868
PSENELK                  866
PNUMEAL                 2239
PNUMENGFL               2239
PNUMUNCFL               2239
PNUMFSM                 2239
PNUMFSMEVER             2241
PERCTOT                 2226
PPERSABS10              2226
TPUP                    2448
P8PUP                   2448
ATT8SCR                 2448
P8MEA                   2448
P8CILOW                 2448
P8CIUPP                 2448
EBACCAPS                2448
PTL2BASICS_94           2448
PTL2BASICS_95           2448
PTEBACC_E_PTQ_EE        2448
PTEBACC_94              2448
PTEBACC_95              2448
PT5EM_94                2448
P8_BANDING              2863
OVERALL_DESTPER         2668
NOT_SUSTAINEDPER        2668
UNKNOWNPER              2668
OVERALL_DESTPER_DIS     2668
NOT_SUSTAINEDPER_DIS    2668
UNKNOWNPER_DIS

* from the above, only four attributes are not missing any data (URN, LA, Gender, Rating)

Reason for having so much of missing data even as expected not to be missing much of it given it is provided from the UK Government website
* from observation most missing data are from features included in the england_census and the key stage 4 datasets and the missing datas are mostly from schools with the school type being **Independent schools** compared with the other school type which are **State-funded** where substantial data are recorded. This is because such schools are not maintained by a local authority and does not have academy status. This type of missing data is the Missing-Not-At-Random (MNAR) pattern.

Options for handling the missing data:
* First option is the deletion of data cases where we have missing feature data so that cases with complete data are used, however this is not the best approach in our case where there is a large number of missing data when compared with the whole data, using this approach means removing above 30% of our data also will result in a bias approach because it will involve removal of the independent schools data from our dataframe.
* Our second option is through Data Imputation via average substitution which involves adding of simulated data (average values) to the missing data feature, however this approach gives room for criticism as it is seen to be manipulated, but also this approach helps to reuce bias when compared with deleting cases where we have missing data.
* Third option is using regression imputation to create a prdictive model which uses available data as regressor to predict values for missing data points

To check for the best approach in handling the missing data, first we tried the average imputation approach below however not all features were handled

In [ ]:
combined_df.fillna(combined_df.mean(), inplace= True)
missing_dt = combined_df.isnull().sum()
missing_dt

Also, we tried the regression imputation approach by creating a linear regression model to fill in missing values, but was unable to successfully create a model because of the present string values (SUPP) in the feautures that couldn't be converted to float values, also there wasn't a feasible deterministic factor to use from the (URN, LA, GENDER, Ratings) features, hence we are going with the first approach below:

In [ ]:
combined_df = combined_df.dropna()
missing_dt = combined_df.isnull().sum()
missing_dt

URN                     0
LA                      0
GENDER                  0
RELCHAR                 0
PNORG                   0
PNORB                   0
PSENELSE                0
PSENELK                 0
PNUMEAL                 0
PNUMENGFL               0
PNUMUNCFL               0
PNUMFSMEVER             0
PNUMFSM                 0
PERCTOT                 0
PPERSABS10              0
TPUP                    0
P8PUP                   0
ATT8SCR                 0
P8MEA                   0
P8CILOW                 0
P8CIUPP                 0
EBACCAPS                0
PTL2BASICS_94           0
PTL2BASICS_95           0
PTEBACC_E_PTQ_EE        0
PTEBACC_94              0
PTEBACC_95              0
PT5EM_94                0
P8_BANDING              0
OVERALL_DESTPER         0
NOT_SUSTAINEDPER        0
UNKNOWNPER              0
OVERALL_DESTPER_DIS     0
NOT_SUSTAINEDPER_DIS    0
UNKNOWNPER_DIS          0
Rating                  0
dtype: int64

In [ ]:
combined_df.head(20)

,URN,LA,GENDER,RELCHAR,PNORG,PNORB,PSENELSE,PSENELK,PNUMEAL,PNUMENGFL,PNUMUNCFL,PNUMFSMEVER,PNUMFSM,PERCTOT,PPERSABS10,TPUP,P8PUP,ATT8SCR,P8MEA,P8CILOW,P8CIUPP,EBACCAPS,PTL2BASICS_94,PTL2BASICS_95,PTEBACC_E_PTQ_EE,PTEBACC_94,PTEBACC_95,PT5EM_94,P8_BANDING,OVERALL_DESTPER,NOT_SUSTAINEDPER,UNKNOWNPER,OVERALL_DESTPER_DIS,NOT_SUSTAINEDPER_DIS,UNKNOWNPER_DIS,Rating
3,100049,202,Mixed,Does not apply,41.4,58.6,3.2,11.8,73.1,26.8,0.1,64.5,41.2,6.9,18.8,199.0,178,42.4,-0.09,-0.28,0.09,3.72,0.588,0.432,0.241,0.191,0.141,0.533,3,0.9,0.09,0.01,0.89,0.11,0.01,Good
4,100050,202,Girls,Does not apply,96.0,4.0,1.3,10.2,38.1,60.9,0.9,49.6,26.9,5.3,12.5,175.0,164,58.6,0.77,0.58,0.97,5.33,0.76,0.577,0.577,0.417,0.326,0.749,1,0.94,0.04,0.02,0.95,0.04,0.01,Good
5,100051,202,Mixed,Does not apply,45.0,55.0,3.6,11.1,82.6,17.4,0.0,69.0,43.0,6.7,19.2,154.0,139,42.1,-0.18,-0.4,0.03,3.77,0.591,0.403,0.604,0.201,0.071,0.526,3,0.86,0.08,0.05,0.85,0.09,0.06,Good
6,100052,202,Mixed,Does not apply,47.5,52.5,2.4,6.9,55.8,44.2,0.0,54.2,22.8,5.4,14,200.0,188,44.6,-0.06,-0.24,0.13,4.08,0.63,0.315,0.585,0.24,0.12,0.57,3,0.91,0.08,0.01,0.87,0.13,0,Good
7,100053,202,Mixed,Does not apply,33.7,66.3,6.2,13.4,28.2,70.3,1.5,47.1,26.2,6.3,18.5,136.0,128,47.2,-0.08,-0.3,0.14,4.29,0.662,0.471,0.515,0.368,0.265,0.625,3,0.95,0.04,0.01,0.9,0.07,0.03,Good
8,100054,202,Girls,None,85.8,14.2,1.5,17.1,36.9,62.3,0.8,36.6,12.9,3.9,3.2,107.0,95,63.4,0.74,0.48,1,5.83,0.888,0.748,0.607,0.551,0.402,0.888,1,0.95,0.04,0.02,0.93,0.05,0.02,Outstanding
9,100055,202,Mixed,Roman Catholic,58.8,41.2,3.5,8.9,52.2,47.8,0.0,54.5,29.3,5.3,14.3,115.0,107,48.3,0.09,-0.15,0.34,4.56,0.67,0.452,0.8,0.409,0.261,0.609,3,0.99,0,0.01,0.98,0,0.02,Good
10,100056,202,Boys,None,7.7,92.3,1.8,6.8,43.7,56.1,0.2,55.0,28.0,5.4,15.6,111.0,105,46.2,-0.44,-0.69,-0.2,4.53,0.658,0.414,0.847,0.396,0.27,0.586,4,0.93,0.06,0.01,0.93,0.07,0,Good
11,100059,202,Girls,Roman Catholic,98.6,1.4,0.8,8.1,43.9,56.0,0.1,42.2,17.3,4.9,9.5,173.0,169,54.6,0.54,0.34,0.73,4.84,0.775,0.532,0.566,0.329,0.214,0.763,1,0.97,0.03,0.01,0.99,0.01,0,Good
25,100092,202,Mixed,Does not apply,11.8,88.2,100.0,0.0,20.6,79.4,0.0,91.2,67.6,27.9,81.4,17.0,16,3.4,-2.98,-3.6,-2.35,0.23,0.059,0,0,0,0,0,5,0.63,0.25,0.13,SUPP,SUPP,SUPP,Good


Findings:
* After observations of the data it appears that some features have the string value SUPP (Suppressed) inputed instead of float/int values and from findings this string is used to indicate that the data at this point is being suppressed in circumstances where there are 5 or fewer students covered by the measure.
* Also there features where the string value NE (No Entry) is inputed instead of float/int values and this is used to indicate that a school or college did not enter any pupils or students for the qualifications covered by the measure.
* Finally, data inputs with the string value LOWCOV (Low Coverage) were observed in place of a float/int value and this is indicated for the ‘value added’ measure and coverage indicator where schools have less than 50% of pupils included in calculation of the measure

**Sanity Checks**

* First sanity check is to confirm that the sum of the percentage number of pupils where english is their first language, those that english isn't their first language and those that whose first language are unclassified should add up to 100 percent and why this is done is because the whole student in the school are being classified into these categories according to the data hence the sum of the number of pupils in each of these categories should add up to the total number of pupils in the school.

In [ ]:
(combined_df["PNUMEAL"] + combined_df["PNUMENGFL"] + combined_df["PNUMUNCFL"]) == 100

3       False
4       False
5        True
6        True
7        True
        ...  
6019     True
6020     True
6021     True
6025     True
6026     True
Length: 3167, dtype: bool

from the check above, some returns False due to rounding error from the initial data input which was rounded to 1 decimal place

* Second sanity check is to check that the percentage of pupils staying in education or employment for at least two terms after key stage 4, and those not staying and those whose activity are not recorded should add to 100 percent since this categorization was done on all the pupils in the school after key stage 4

In [ ]:
(combined_df["OVERALL_DESTPER_DIS"] + combined_df["NOT_SUSTAINEDPER_DIS"] + combined_df["UNKNOWNPER_DIS"]) == 1

running the above check expected some errors some of the input values for these feature category has string inputs (SUPP) which cannot be added. To further achieve this check, random samples can be gotten from the dataframe to check the above which is an approach to sanity check

In [ ]:
sample_data = combined_df.sample(n=10)
(sample_data["OVERALL_DESTPER_DIS"] + sample_data["NOT_SUSTAINEDPER_DIS"] + sample_data["UNKNOWNPER_DIS"]) == 1

3018    True
4348    True
5440    True
334     True
1302    True
3081    True
2517    True
5275    True
3443    True
37      True
dtype: bool

* Exporting the final clean data for further use

In [ ]:
combined_df.to_excel("Clean Combined OFSTED.xlsx")